# 0) 설치 & 기본 설정

In [ ]:
# (Colab 필요 시) 시스템/라이브러리 설치
!apt-get -qq install pv
!pip -q install rasterio matplotlib numpy
!pip install transformers torch rasterio
!pip install opencv-python-headless
!pip install --upgrade transformers datasets accelerate

In [ ]:
# ---- Imports & 전역 옵션 ----
import os, re, time, tarfile, subprocess, shutil
import numpy as np
import cv2
import rasterio

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor

# Drive 마운트: 체크포인트/아웃풋 저장용
from google.colab import drive
drive.mount('/content/drive')

# 성능 최적화: cudnn 커널 탐색 허용, matmul 정밀도 완화(지원 시)
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("medium")
except Exception:
    pass

# 1) 데이터 압축 해제 파이프라인
여러 개로 분할된 *.zip.part*를 정렬 → 병합 → unzip하는 자동화.

subprocess.run() 으로 셸 호출을 안전하게 수행.


In [ ]:
def run_cmd(cmd_list):
    """외부 명령 실행(실패 시 예외 발생)"""
    subprocess.run(cmd_list, check=True)

def final_extract_and_rename_pythonic(tar_path, output_folder):
    """
    입력: .tar 경로 / 출력: 압축 해제된 파일(이미지 또는 라벨)
    단계:
      1) tar 풀기
      2) 내부의 *.zip.part* 존재 시 순서대로 rename
      3) part들을 이어붙여 하나의 zip으로 병합
      4) unzip 진행
      5) 임시 디렉터리 정리
    """
    base_name = os.path.basename(tar_path)
    print(f"🚀 Processing {base_name}...")
    start_time = time.time()

    # 임시 디렉터리: 파일 충돌 방지를 위해 tar 파일명 기반으로 생성
    temp_dir = f"/content/temp_{os.path.splitext(base_name)[0]}"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir, ignore_errors=True)
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(output_folder, exist_ok=True)

    # 1) .tar 풀기
    print(f"[{base_name}] Step 1: Un-tarring...")
    try:
        with tarfile.open(tar_path, 'r') as tar:
            tar.extractall(path=temp_dir)
    except tarfile.TarError as e:
        print(f"[{base_name}] tar extracting failed: {e}")
        shutil.rmtree(temp_dir, ignore_errors=True)
        return
    print(f"[{base_name}] Un-tarring complete.")

    # 2) .zip.part* 찾기 (없는 경우 단일 zip 여부 확인)
    print(f"[{base_name}] Step 2: Finding and renaming part files...")
    part_files = [os.path.join(root, f)
                  for root, _, files in os.walk(temp_dir)
                  for f in files if '.zip.part' in f]

    def get_part_number(filepath):
        """파일명 끝의 .partNN 숫자만 추출하여 정렬 키로 사용"""
        try:
            return int(re.search(r'\.part(\d+)$', os.path.basename(filepath)).group(1))
        except (AttributeError, ValueError):
            return -1
    part_files.sort(key=get_part_number)

    if not part_files:
        # part가 없으면 단일 zip만 존재할 수 있음
        zip_files = [os.path.join(root, f)
                     for root, _, files in os.walk(temp_dir)
                     for f in files if f.endswith('.zip')]
        if zip_files:
            print(f"[{base_name}] Unzipping single zip...")
            run_cmd(["unzip", "-o", "-q", zip_files[0], "-d", output_folder])
        else:
            print(f"[{base_name}] No zip files to extract.")
    else:
        # part가 있으면: 일관된 이름으로 재명명 후 zip 병합
        zip_file_basename = ""
        renamed_paths = []
        match = re.match(r'(.+\.zip)\.part.*', os.path.basename(part_files[0]))
        if match:
            zip_file_basename = match.group(1)
            dir_name = os.path.dirname(part_files[0])
            # .part00, .part01 ... 로 강제 정렬(누락·순서틀림 방지)
            for i, old_path in enumerate(tqdm(part_files, desc=f"[{base_name}] Renaming", leave=False)):
                new_path = os.path.join(dir_name, f"{zip_file_basename}.part{i:02d}")
                os.rename(old_path, new_path)
                renamed_paths.append(new_path)

        if not renamed_paths:
            print(f"[{base_name}] ❌ Error: Renaming failed.")
        else:
            print(f"[{base_name}] Renamed {len(part_files)} files.")
            # 3) 파트를 차례대로 이어붙여 단일 zip 생성
            print(f"[{base_name}] Step 3: Combining and Unzipping...")
            parts_directory = os.path.dirname(renamed_paths[0])
            combined_zip_path = os.path.join(parts_directory, zip_file_basename)
            with open(combined_zip_path, "wb") as w:
                for p in sorted(renamed_paths):
                    with open(p, "rb") as r:
                        shutil.copyfileobj(r, w)
            # 4) unzip 실행
            run_cmd(["unzip", "-o", "-q", combined_zip_path, "-d", output_folder])

    # 5) 임시 파일 정리(스토리지 절약)
    print(f"[{base_name}] Step 4: Cleaning up temporary files...")
    shutil.rmtree(temp_dir, ignore_errors=True)

    end_time = time.time()
    print(f"✅ Done with {base_name} in {end_time - start_time:.2f} seconds.")
    print("-" * 40)

# (필요시) 병렬로 호출
import concurrent.futures
tasks = [
     ("/content/drive/MyDrive/VS_SN.tar", "/content/vs_sn"),
     ("/content/drive/MyDrive/TL_SN.tar", "/content/tl_sn"),
     ("/content/drive/MyDrive/VL_SN.tar", "/content/vl_sn"),
     ("/content/drive/MyDrive/TS_SN.tar", "/content/ts_sn")
 ]
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
     executor.map(lambda p: final_extract_and_rename_pythonic(*p), tasks)
print("\n🎉 All datasets processing finished!")

# 2) 데이터셋 & 전처리

이미지: (C,H,W) uint16 → /256으로 uint8 스케일 → (H,W,C)로 변환 후 AREA 리사이즈 → (C,H,W) 복귀 → float32/255로 [0,1] 정규화


라벨: (H,W) → NEAREST 리사이즈 → 90→1 이진화(그 외 0)

In [ ]:
class SegformerDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_processor):
        # 숨김파일 제외 + 이름 정렬(이미지·라벨 1:1 매칭을 위해 중요)
        self.image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if not f.startswith('.')])
        self.label_paths = sorted([os.path.join(label_dir, f) for f in os.listdir(label_dir) if not f.startswith('.')])

        # 개수 불일치 방지(초기에 잡아야 디버그 쉬움)
        assert len(self.image_paths) == len(self.label_paths), \
            f"Count mismatch: {len(self.image_paths)} images vs {len(self.label_paths)} labels"

        # (선택) 파일명 스템 일치 여부를 엄격 검사(라벨-이미지 뒤섞임 방지)
        for ip, lp in zip(self.image_paths, self.label_paths):
            if os.path.splitext(os.path.basename(ip))[0] != os.path.splitext(os.path.basename(lp))[0]:
                raise ValueError(f"Name mismatch: {os.path.basename(ip)} vs {os.path.basename(lp)}")

        self.image_processor = image_processor
        # SegFormer 이미지 크기 설정을 그대로 사용(height==width 가정)
        self.target_size = self.image_processor.size['height']

    def __getitem__(self, idx):
        # --- 이미지 읽기 (uint16) ---
        with rasterio.open(self.image_paths[idx]) as src:
            image = src.read()                     # (C,H,W), uint16

        # uint16 → uint8 스케일 다운: Sentinel-2 대역을 간결히 8비트로
        image_rescaled = (image / 256).astype(np.uint8)

        # 리사이즈를 위해 (H,W,C)로 전치
        image_hwc = np.transpose(image_rescaled, (1, 2, 0))
        resized_image = cv2.resize(
            image_hwc, (self.target_size, self.target_size),
            interpolation=cv2.INTER_AREA           # 연속톤 이미지에 적합
        )

        # 모델 입력 형태(C,H,W)로 복귀 후 [0,1] 정규화 (float32)
        image_chw = np.transpose(resized_image, (2, 0, 1))
        pixel_values = torch.from_numpy(image_chw).float() / 255.0

        # --- 라벨 읽기 & 이진화 ---
        with rasterio.open(self.label_paths[idx]) as src:
            label = src.read(1)                    # (H,W)

        # 클래스 경계 보존을 위해 최근접 보간
        resized_label = cv2.resize(
            label, (self.target_size, self.target_size),
            interpolation=cv2.INTER_NEAREST
        )

        # 값 90만 타깃(1), 나머지 0
        new_label = np.zeros_like(resized_label, dtype=np.int64)
        new_label[resized_label == 90] = 1

        return {"pixel_values": pixel_values, "labels": torch.from_numpy(new_label).long()}

    def __len__(self):
        return len(self.image_paths)


# 3) 모델 빌더 — SegFormer-B1 (4채널 입력 지원)
HuggingFace SegformerForSemanticSegmentation 로드

첫 patch embedding conv(in_channels: 3)를 4채널로 교체 + 가중치 이식

4번째 채널은 기존 3채널 평균으로 초기화(안정적 시작)

In [ ]:
def build_segformer_4ch(num_classes=2, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = SegformerForSemanticSegmentation.from_pretrained(
        "nvidia/segformer-b1-finetuned-ade-512-512",
        num_labels=num_classes,                    # 출력 클래스(배경/산업단지)
        ignore_mismatched_sizes=True               # 헤드 크기 불일치 허용
    )

    # --- 3→4채널 conv 교체 ---
    original_conv = model.segformer.encoder.patch_embeddings[0].proj
    old_weights = original_conv.weight.clone()     # (out, 3, k, k)
    new_conv = nn.Conv2d(
        4, original_conv.out_channels,
        kernel_size=original_conv.kernel_size,
        stride=original_conv.stride,
        padding=original_conv.padding
    )
    with torch.no_grad():
        # 기존 3채널은 그대로 복사
        new_conv.weight[:, :3, :, :] = old_weights
        # 4번째 채널은 3채널 평균으로 초기화(복제 대비 안정적)
        new_conv.weight[:, 3:, :, :] = old_weights.mean(1, keepdim=True)
        if original_conv.bias is not None:
            new_conv.bias = nn.Parameter(original_conv.bias.clone())
    model.segformer.encoder.patch_embeddings[0].proj = new_conv

    model.to(device)

    # (선택) 커널 퓨전/그래프 최적화: 지원되지 않으면 무시
    try:
        model = torch.compile(model)
    except Exception:
        pass
    return model


# 4) 평가 함수
4-1. 공식 — sigmoid(z1 - z0) 기반 Accuracy / IoU@0.5

2-클래스 로짓 z0,z1에서 차이를 취해 안정적인 확률 계산

확률 0.5 임계값으로 이진 마스크 생성 → IoU 계산

In [ ]:
@torch.inference_mode()
def evaluate_sigmoid(model, dataloader, device):
    """
    공식 평가:
      - logits -> bilinear 업샘플링(H,W 정합)
      - z1 - z0 -> sigmoid -> prob1
      - pred = (prob1 > 0.5)
      - Accuracy, IoU(클래스=1 기준) 반환
    """
    model.eval()
    total_correct = 0
    total_pixels  = 0
    total_inter   = 0
    total_union   = 0

    for batch in tqdm(dataloader, desc="[Evaluation]"):
        pv = batch["pixel_values"].to(device, non_blocking=True)   # (B,4,H,W)
        gt = batch["labels"].to(device, non_blocking=True)         # (B,H,W), 0/1

        out = model(pixel_values=pv)                               # logits: (B,2,h,w)
        logits = F.interpolate(out.logits, size=gt.shape[-2:], mode='bilinear', align_corners=False)

        logit_diff = logits[:, 1] - logits[:, 0]                   # (B,H,W)
        prob1 = torch.sigmoid(logit_diff)                          # (B,H,W), target 확률
        pred  = (prob1 > 0.5).long()                               # (B,H,W), 0/1

        total_correct += (pred == gt).sum().item()
        total_pixels  += gt.numel()

        inter = ((pred == 1) & (gt == 1)).sum().item()
        union = ((pred == 1) | (gt == 1)).sum().item()
        total_inter += inter
        total_union += union

    acc = total_correct / total_pixels
    iou = (total_inter / total_union) if total_union > 0 else 0.0
    return acc, iou


4-2. 참고 — softmax-argmax 기반 mIoU/PixelAcc

클래스별 IoU 평균(mIoU)을 계산할 때 유용(보고용)

In [ ]:
@torch.inference_mode()
def evaluate_argmax_miou(model, dataloader, device, num_classes=2):
    """
    참고 지표:
      - softmax/argmax로 클래스 예측(0/1)
      - 클래스별 IoU 평균(mIoU) + Pixel Accuracy 반환
    """
    model.eval()
    all_preds, all_labels = [], []
    for batch in tqdm(dataloader, desc="[Eval-argmax]"):
        pv = batch["pixel_values"].to(device, non_blocking=True)
        gt = batch["labels"].to(device, non_blocking=True)
        out = model(pixel_values=pv)
        logits = F.interpolate(out.logits, size=gt.shape[-2:], mode='bilinear', align_corners=False)
        preds = torch.argmax(logits, dim=1)         # (B,H,W)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(gt.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    iou_per_class = []
    for cls in range(num_classes):
        inter = np.sum((all_preds == cls) & (all_labels == cls))
        uni   = np.sum((all_preds == cls) | (all_labels == cls))
        if uni > 0:
            iou_per_class.append(inter / uni)

    macro_miou = float(np.mean(iou_per_class)) if iou_per_class else 0.0
    micro_acc  = float(np.sum(all_preds == all_labels) / all_labels.size)
    return macro_miou, micro_acc


# 5) 설정값 & 경로


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습/평가 스위치: 학습은 True, 저장된 체크포인트로 평가만 할 땐 False
TRAIN = True

# 하이퍼파라미터
EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 5e-5

# 체크포인트 경로(Drive 저장 권장)
CHECKPOINT = "/content/drive/MyDrive/M3_final_seg.pth"

# 데이터 경로(압축 해제 결과)
TR_IMG_DIR = "/content/ts_sn"
TR_LAB_DIR = "/content/tl_sn"
VAL_IMG_DIR = "/content/vs_sn"
VAL_LAB_DIR = "/content/vl_sn"


# 6) 데이터로더 준비


In [ ]:
image_processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512")

train_dataset = SegformerDataset(TR_IMG_DIR, TR_LAB_DIR, image_processor)
val_dataset   = SegformerDataset(VAL_IMG_DIR, VAL_LAB_DIR, image_processor)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, pin_memory=True, persistent_workers=True  # 워커 유지로 초기화 비용 절감
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True, persistent_workers=True
)


# 7) 모델 생성 & (옵션) 체크포인트 로드


In [ ]:
model = build_segformer_4ch(num_classes=2, device=device)

# 평가만 진행하고 싶다면 TRAIN=False로 두고 체크포인트 로드
if not TRAIN and os.path.exists(CHECKPOINT):
    model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
    model.eval()
    print("✅ Loaded checkpoint for evaluation only.")


# 8) 학습 루프 & 검증(공식/참고 지표 동시 출력)

In [ ]:
if TRAIN:
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.amp.GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

    print("----------- 학습 시작 -----------")
    best_iou = -1.0  # 공식 IoU(sigmoid@0.5) 기준으로 최고값 추적

    for epoch in range(1, EPOCHS+1):
        model.train()
        epoch_loss = 0.0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [Train]"):
            pv = batch["pixel_values"].to(device, non_blocking=True)
            gt = batch["labels"].to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            # AMP: 혼합정밀로 연산 속도/메모리 절약
            with torch.amp.autocast('cuda' if torch.cuda.is_available() else 'cpu'):
                out = model(pixel_values=pv, labels=gt)  # 내부 CE loss 사용
                loss = out.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / max(1, len(train_loader))

        # 참고용: argmax mIoU(보고서용 표기)
        miou_argmax, micro_acc_argmax = evaluate_argmax_miou(model, val_loader, device, num_classes=2)
        # 공식: sigmoid@0.5 Acc/IoU
        val_acc, val_iou = evaluate_sigmoid(model, val_loader, device)

        print(f"Epoch {epoch}/{EPOCHS} | Loss: {avg_loss:.4f} | "
              f"mIoU(argmax): {miou_argmax:.4f} | Acc(sigmoid@0.5): {val_acc:.4f} | IoU(sigmoid@0.5): {val_iou:.4f}")

        # **공식 IoU@0.5** 최고치 기준으로 체크포인트 저장
        if val_iou > best_iou:
            best_iou = val_iou
            torch.save(model.state_dict(), CHECKPOINT)
            print(f"🏆 New best (IoU@0.5) saved at {CHECKPOINT}: {best_iou:.4f}")

    print("----------- 학습 종료 -----------")
    print(f"최고 IoU@0.5: {best_iou:.4f}")


# 9) 최종 평가(공식 지표)

In [ ]:
print("----------- 최종 평가 (sigmoid@0.5) -----------")
model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
model.eval()
val_acc, val_iou = evaluate_sigmoid(model, val_loader, device)
print(f"📊 Final Validation — Acc: {val_acc:.4f} | IoU@0.5(sigmoid): {val_iou:.4f}")
